In [23]:
using Zygote
using Flux
using CuArrays

In [1]:
push!(LOAD_PATH, ".")

4-element Array{String,1}:
 "@"      
 "@v#.#"  
 "@stdlib"
 "."      

In [8]:
using Setup: *

In [11]:
using Zygote
using Flux
using CuArrays

In [71]:
Zygote.@adjoint gpu(a :: Array) = gpu(a), a̅ -> (cpu(a̅),)
Zygote.@adjoint Base.collect(a :: CuArray) = Base.collect(a), a̅ -> (gpu(a̅),)
Zygote.refresh()

In [72]:
gradient(x -> reduce(*, gpu(x), init=1.0), rand(3, 3))

ErrorException: Can't differentiate gc_preserve_end expression

In [81]:
@code_typed reduce(*, gpu(rand(3, 3)), init=1.0)

CodeInfo(
│      1 ──       goto #3 if not false
│      2 ──       nothing::Nothing
│      3 ── %3  = CuArrays.:(:)::Const(Colon(), false)
│╻     getindex │    %4  = (Base.getfield)(#temp#, :init)::Float64
││╻╷╷╷  iterate │    %5  = (Base.sle_int)(1, 1)::Bool
│││┃│    iterate └───       goto #5 if not %5
││││┃│    iterate 4 ── %7  = (Base.sle_int)(1, 0)::Bool
│││││  └───       goto #6
│      5 ──       nothing::Nothing
│││││  6 ┄─ %10 = φ (#4 => %7, #5 => false)::Bool
│││││  └───       goto #8 if not %10
│││││  7 ──       invoke Base.getindex(()::Tuple{}, 1::Int64)::Union{}
│││││  └───       $(Expr(:unreachable))::Union{}
│││││  8 ──       goto #10
│││││  9 ──       $(Expr(:unreachable))::Union{}
││││   10 ┄       goto #11
││╻     iterate 11 ─       goto #12
││     12 ─       goto #13
│      13 ─       nothing::Nothing
│╻     #reduce#59 │    %20 = invoke CuArrays._reduce(_4::Function, _5::CuArray{Float32,2,Nothing}, %4::Float64, %3::Colon)::Any
│      └───       return %20
) => Any

In [153]:
Flux.@progress for i in 1:10
    ()
end

In [156]:
@info "hello"
sleep(5.0)
@info "goodbye"

┌ Info: hello
└ @ Main In[156]:1
┌ Info: goodbye
└ @ Main In[156]:3


In [ ]:
@info("Beginning training loop...")
sleep(100)

┌ Info: Beginning training loop...
└ @ Main In[173]:1


In [14]:
?Flux.@progress

```
@progress [name="", threshold=0.005] for i = ..., j = ..., ...
@progress [name="", threshold=0.005] x = [... for i = ..., j = ..., ...]
```

Show a progress meter named `name` for the given loop or array comprehension if possible. Update frequency is limited by `threshold` (one update per 0.5% of progress by default).


In [26]:
@showprogress for i in 1:100
    sleep(0.05)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:05


```
@showprogress dt "Computing..." for i = 1:50
    # computation goes here
end

@showprogress dt "Computing..." pmap(x->x^2, 1:50)
```

displays progress in performing a computation. `dt` is the minimum interval between updates to the user. You may optionally supply a custom message to be printed that specifies the computation being performed.

`@showprogress` works for loops, comprehensions, map, and pmap.


In [62]:
gradient(x -> sum(cpu(x)), gpu(rand(3, 3)))

┌ Warning: Performing scalar operations on GPU arrays: This is very slow, consider disallowing these operations with `allowscalar(false)`
└ @ GPUArrays /data/scratch/jhgilles/.julia/packages/GPUArrays/1wgPO/src/indexing.jl:16


(Float32[1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0],)

```
copyto!(dest, do, src, so, N)
```

Copy `N` elements from collection `src` starting at offset `so`, to array `dest` starting at offset `do`. Return `dest`.

---

```
copyto!(dest::AbstractArray, src) -> dest
```

Copy all elements from collection `src` to array `dest`, whose length must be greater than or equal to the length `n` of `src`. The first `n` elements of `dest` are overwritten, the other elements are left untouched.

# Examples

```jldoctest
julia> x = [1., 0., 3., 0., 5.];

julia> y = zeros(7);

julia> copyto!(y, x);

julia> y
7-element Array{Float64,1}:
 1.0
 0.0
 3.0
 0.0
 5.0
 0.0
 0.0
```

---

```
copyto!(dest, Rdest::CartesianIndices, src, Rsrc::CartesianIndices) -> dest
```

Copy the block of `src` in the range of `Rsrc` to the block of `dest` in the range of `Rdest`. The sizes of the two regions must match.


In [59]:
?CUDAdrv.unsafe_copyto!

```
unsafe_copyto!(dest::Ptr{T}, src::Ptr{T}, N)
```

Copy `N` elements from a source pointer to a destination, with no checking. The size of an element is determined by the type of the pointers.

The `unsafe` prefix on this function indicates that no validation is performed on the pointers `dest` and `src` to ensure that they are valid. Incorrect usage may corrupt or segfault your program, in the same manner as C.

---

```
unsafe_copyto!(dest::Array, do, src::Array, so, N)
```

Copy `N` elements from a source array to a destination, starting at offset `so` in the source and `do` in the destination (1-indexed).

The `unsafe` prefix on this function indicates that no validation is performed to ensure that N is inbounds on either array. Incorrect usage may corrupt or segfault your program, in the same manner as C.


In [64]:
@show

In [70]:
?Base.collect

```
collect(element_type, collection)
```

Return an `Array` with the given element type of all items in a collection or iterable. The result has the same shape and number of dimensions as `collection`.

# Examples

```jldoctest
julia> collect(Float64, 1:2:5)
3-element Array{Float64,1}:
 1.0
 3.0
 5.0
```

---

```
collect(collection)
```

Return an `Array` of all items in a collection or iterator. For dictionaries, returns `Pair{KeyType, ValType}`. If the argument is array-like or is an iterator with the [`HasShape`](@ref IteratorSize) trait, the result will have the same shape and number of dimensions as the argument.

# Examples

```jldoctest
julia> collect(1:2:13)
7-element Array{Int64,1}:
  1
  3
  5
  7
  9
 11
 13
```


In [ ]:
]add ProgressMeter